# Combining and analysing the low density anglescan simulations

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu

In [2]:
importlib.reload(spl)
importlib.reload(td)
importlib.reload(hmg)
importlib.reload(fd)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/Coding/Projects/flopter/flopter/core/lputils.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [3]:
# lowdens_dir = pth.Path('/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan')
lowdens_dir = pth.Path('/home/jleland/Data/external/spice/')
os.chdir(lowdens_dir)

In [5]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [6]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/rearwall_tiltscan',
    'marconi/lowdens_anglescan/recessed_tiltscan'
}

In [7]:
scans_searchstr = '*/lowdens_anglescan/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in skippable_scans:
        print(f'Skipping {scan}...\n\n')
        continue
    all_run_dirs[scan] = glob.glob(scan + angles_search_str)
#     all_run_dirs[scan] = list(range(len(temp)))
#     for d in temp:
#         i = int(np.abs(float(d[-4:]))) - 1
#         print(i)
#         all_run_dirs[scan][i] = d
    
#     all_run_dirs[scan].sort()
    
    print(f'{scan}: {all_run_dirs[scan]}\n')
    


cumulus/lowdens_anglescan/flush_tiltscan_redo: ['cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-1.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-10.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-15.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-2.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-20.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-25.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-3.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-30.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-45.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-60.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-75.0', 'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-90.0']

cumulus/lowdens_anglescan/flush_tiltscan_ext: ['cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-1.0', 'cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-70.0', 'cumulus/lowde

---
## Fucking around
This is a testbed for ideas implemented in splopter's init and later in the notebook. This is not necessary to be run.

In [10]:
scan = scans[1]
filename = lowdens_dir / all_run_dirs[scan][0] / 't-alpha_yz_-9.0.mat'
# filename = lowdens_dir / all_run_dirs[scan][0]
t_file_info = sio.whosmat(filename)
# for info in t_file_info:
#     print(info)
    
# t_file_labels, t_file_sizes, t_file_types = zip(*t_file_info)
# t_file_labels, t_file_sizes, t_file_types = zip(*sio.whosmat(filename))
t_file_sizes = {label: size for label, size, dtype in sio.whosmat(filename)}
t_file_labels = set(t_file_sizes.keys())
# print(t_file_labels)

for var in desired_variables:
    if var in t_file_labels:
        print(f'{var}: {t_file_sizes[var]}')
    else:
        print(f'WARNING: {var}')


alphaxz: (1, 1)
q: (2, 1)
ProbePot: (416340, 1)
dz: (1, 1)
objectscurrenti: (4, 1219)
objectspowerfluxe: (4, 1219)
Pot: (191, 557)
objectscurrente: (4, 1219)
t: (1220, 1)
Potvac: (191, 557)
m: (2, 1)
Temp: (2, 1)
objectspowerfluxi: (4, 1219)
objectsenum: (191, 557)
alphayz: (1, 1)
dt: (1, 1)


In [7]:
filename = lowdens_dir / all_run_dirs[scan][0]
backups = [] + list(filename.glob('backup_*'))
backups.sort()

print(filename)
print(len(backups))

/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan/angled_tiltscan/alpha_yz_-9.0
2


In [11]:
prel_vars = ['t', 'dt']
prel_tfile = sio.loadmat(filename, variable_names=prel_vars)
print([prel_tfile[var] for var in prel_vars])
print(np.mean(prel_tfile['t']))
print(len(prel_tfile['t']))
synth_t = np.arange(1, len(prel_tfile['t']) + 1) * 2048.0 * prel_tfile['dt']
print(synth_t - np.squeeze(prel_tfile['t']))

[array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), array([[0.00033962]])]
0.0
1220
[[6.95532042e-01 1.39106408e+00 2.08659612e+00 ... 8.47158027e+02
  8.47853559e+02 8.48549091e+02]]


---
## Single splopter plotting and fitting

In [29]:
chosen_spl_path = lowdens_dir / all_run_dirs[scan][2]
print(chosen_spl_path)
splopter = spl.Splopter(chosen_spl_path, reduce=desired_variables)
splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
individual_spl_label = all_run_dirs['cumulus/lowdens_anglescan/angled_tiltscan_redo'][0]
print(individual_spl_label)
splopter = splopters[individual_spl_label]

cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0


NameError: name 'splopters' is not defined

In [16]:
fig, ax = plt.subplots()
print(splopter.tdata.t_dict['t'])
ax.plot(splopter.raw_data['t'][:-2], splopter.raw_data['I'][:-2])
ax.plot(splopter.raw_data['t'][:-2], splopter.raw_data['V'][:-2])

[[1.73883010e-01]
 [3.47766021e-01]
 [5.21649031e-01]
 ...
 [1.90749662e+02]
 [1.90923545e+02]
 [1.91097428e+02]]


In [17]:
fig, ax = plt.subplots()
ax.plot(splopter.iv_data['I'][:-2])

In [18]:
iv_region = np.where(splopter.iv_data['V'] < -2)
print(iv_region)

fig, ax = plt.subplots()
ax.plot(splopter.iv_data['V'][iv_region], splopter.iv_data['I'][iv_region])

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144]),)


In [19]:
fig, ax = plt.subplots()
ax.plot(np.diff(splopter.iv_data['V'][iv_region]), '.')

In [22]:
i = np.where((np.diff(splopter.iv_data['V'][iv_region]) == 0))
i = (i[0] + 1, )

print(len(i[0]) / len(iv_region[0]), len(i))

if len(i[0]) / len(iv_region[0]) <= 0.25:
    print('Big downsample, skipping...')
    i = iv_region

fig, ax = plt.subplots(2)
ax[0].plot(splopter.iv_data['t'][iv_region], splopter.iv_data['V'][iv_region])
ax[0].plot(splopter.iv_data['t'][i], splopter.iv_data['V'][i], '.')

ax[1].plot(splopter.iv_data['V'][iv_region], splopter.iv_data['I'][iv_region])
ax[1].plot(splopter.iv_data['V'][i], splopter.iv_data['I'][i], '.')

0.0 1
Big downsample, skipping...


In [23]:
fig, ax = plt.subplots(2)
ax[0].plot(splopter.iv_data['V'][iv_region], splopter.iv_data['I'][iv_region])
ax[1].plot(splopter.iv_data['V'][i], splopter.iv_data['I'][i])

In [24]:
iv_data_trimmed = iv.IVData.non_contiguous_trim(splopter.iv_data, i)

plt.figure()
plt.plot(iv_data_trimmed['V'], iv_data_trimmed['I'])

fitter = fts.FullIVFitter()
fit_data = fitter.fit_iv_data(iv_data_trimmed)
fit_data.plot()

## Plotting all IV characterisitics on the same axes

In [68]:
skippable = {
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-3.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0_restart',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-90.0',
    'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-1.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-6.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-7.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-8.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-9.0',
#     'cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-1.0',
#     'cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0'
}

In [69]:
# fig, axes = plt.subplots(2)
splopters = {}
scans = list(scans)
for i in range(len(scans)):
#     ax = axes[i]
    scan = scans[i]

    for angle_dir in all_run_dirs[scan]:
        print(f'\nCreating splopter for {scan}, {angle_dir}')
        if angle_dir in skippable:
            print('Skipping...')
            continue
        splopter = spl.Splopter(lowdens_dir / angle_dir, reduce=desired_variables)
        splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
#         splopter.denormalise()
        splopters[angle_dir] = splopter
#         ax.plot(splopter.iv_data['V'][:320], splopter.iv_data['I'][:320])
        
print('\n...Done')


Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-1.0
Skipping...

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-2.0
Skipping...

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-3.0
Skipping...

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-4.0


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-5.0

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-6.0

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-7.0

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-8.0

Creating splopter for marconi/lowdens_anglescan/inpartest_as_restart, marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-9.0

Creating splopter for marconi/lowdens_anglescan/angled_tiltscan_ext, marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0

Creating splopter for marconi/lowdens_anglescan/angled_tiltscan_ext, marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-70.0

Creating splopter for marconi/lowdens_anglescan/angled_tiltscan_ext, marconi/l

In [11]:
# For adding a new run to the list of splopters
angle_dir = all_run_dirs[scans[6]][0]
print(angle_dir)

if angle_dir in splopters:
    print(f'Removing {angle_dir} from existing list of splopters')
    splopters.pop(angle_dir)
splopter = spl.Splopter(lowdens_dir / angle_dir, reduce=desired_variables)
splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
splopter.denormalise()
splopters[angle_dir] = splopter

marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-1.0


In [26]:
print(splopters.keys())
print(len(splopters.keys()))

dict_keys(['marconi/lowdens_anglescan/inpartest_as/alpha_yz_-9.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-1.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-2.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-4.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-7.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-8.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-6.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-3.0', 'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-5.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-9.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-1.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-2.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-4.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-10.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-3.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-5.0', 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-6.0', 'marconi/low

In [21]:
padder = np.zeros([11, 5])

fit_df = pd.DataFrame(columns=['probe', 'angle', 'temp', 'isat', 'a', 'v_f'])
# fit_df = fit_df.append({'probe':'flush', 'angle':0.0, 'temp':1.0, 'isat':1.0, 'a':0.002, 'v_f':-3}, ignore_index=True)
# fit_df = fit_df.append({'probe':'angled', 'angle':0.0, 'temp':1.0, 'isat':1.0, 'a':0.002, 'v_f':-3}, ignore_index=True)
fit_df.where(fit_df.probe == 'angled').dropna()
fit_df

angles    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
dtype: object
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


,probe,angle,temp,isat,a,v_f


In [7]:
colourmap = plt.get_cmap('nipy_spectral')
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[colourmap(k) for k in np.linspace(0, 1, 12)])

In [74]:
fig, axes = plt.subplots(2, 3, sharex='col', sharey='col')

fit_df = pd.DataFrame(columns=['probe', 
                               'angle', 
                               'temp',
                               'd_temp',
                               'isat', 
                               'd_isat',
                               'a',
                               'd_a',
                               'v_f',
                               'd_v_f'
                              ])

for label, s in splopters.items():
    print('=== ' + label + ' ===')
#     if angle == 2.0:
#         continue
    
    if label in skippable:
        print(f'Skipping {label}')
        continue
    elif 'inpartest' in label or 'flush' in label:
        ax = axes[0][0]
        axt = axes[0][1]
        axd = axes[0][2]
        probe = 'flush'
        ax.set_title('Flush')
    else:
        ax = axes[1][0]
        axt = axes[1][1]
        axd = axes[1][2]
        probe = 'angled'
        ax.set_title('Angled Tip')
        
    angle = np.abs(float(label[-4:]))
    
    iv_region = np.where(s.iv_data['V'] < -2)
    i = np.where((np.diff(s.iv_data['V'][iv_region]) == 0))
    i = (i[0] + 1, )
    
#     if len(i[0]) / len(iv_region[0]) <= 0.25:
#         print('Big downsample, skipping...')
    i = iv_region
#     s.denormalise()
    
    ax.errorbar(s.iv_data['V'][i], s.iv_data['I'][i], label=label[-4:], yerr=s.iv_data['sigma'][i])
    ax.legend()
    
    try:
        fit_iv_data = iv.IVData.non_contiguous_trim(s.iv_data, i)
        fitter = fts.FullIVFitter()
        fit_data = fitter.fit_iv_data(fit_iv_data)
        #     ax.plot(*fit_data.get_fit_plottables())

        fit_df = fit_df.append({'probe': probe, 
                                'angle': angle, 
                                'temp': fit_data.get_temp(),
                                'd_temp': fit_data.get_temp_err(),
                                'isat': fit_data.get_isat(), 
                                'd_isat': fit_data.get_isat_err(),
                                'a': fit_data.get_sheath_exp(), 
                                'd_a': fit_data.get_sheath_exp_err(),
                                'v_f': fit_data.get_floating_pot(),
                                'd_v_f': fit_data.get_floating_pot_err()}, ignore_index=True)

        axt.errorbar(angle, fit_data.get_temp(), yerr=fit_data.get_temp_err(), fmt='x') #, ecolor='silver')
        axd.errorbar(angle, fit_data.get_isat(), yerr=fit_data.get_isat_err(), fmt='x') #, ecolor='silver')
    except RuntimeError as e:
        print(f'WARNING: Encountered error in fit, skipping {label}')

for j, probe in enumerate(['flush', 'angled']):
    filt_df = fit_df.where(fit_df.probe == probe).dropna()
    axt = axes[j][1]
    axd = axes[j][2]
    if j == 0:
        axt.set_title(r'$T_e$')
        axd.set_title(r'$I_{sat}$')
    if j == 1:
        axt.set_xlabel(r'$\theta$')
        axd.set_xlabel(r'$\theta$')
#     axt.errorbar(filt_df['angle'], filt_df['temp'] - 1, yerr=filt_df['d_temp'], fmt='x', ecolor='silver')
#     axt.set_ylim(0.3, 1.7)
    axt.axhline(y=1.0, color='k', linewidth=0.5, linestyle='dotted')
#     axd.errorbar(filt_df['angle'], filt_df['isat'], yerr=filt_df['d_isat'], fmt='x', ecolor='silver')
#     axd.set_ylim(-0.1, 90)
    
fit_df


=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-4.0 ===
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-5.0 ===
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-6.0 ===
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-7.0 ===
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-8.0 ===
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-9.0 ===
=== marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0 ===
=== marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-70.0 ===
=== marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-80.0 ===
=== marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-85.0 ===
=== marconi/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-89.0 ===
=== cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-70.0 ===
=== cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-80.0 ===
=== cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-85.0 ===
=== cumulus/lowdens_anglescan/angle

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,flush,4.0,0.862937,0.021906,8.867943,0.236408,1.305785e-01,0.008518,-3.000390,0.007916
1,flush,5.0,0.984287,0.023889,12.246094,0.301755,8.804362e-02,0.007808,-2.990635,0.006228
2,flush,6.0,0.980118,0.021951,14.471464,0.340238,7.609351e-02,0.007440,-3.051633,0.006168
3,flush,7.0,0.940242,0.009109,16.238058,0.168016,7.587003e-02,0.003233,-3.054091,0.002874
4,flush,8.0,1.017323,0.009001,19.388100,0.176069,5.588848e-02,0.002833,-3.014263,0.002248
5,flush,9.0,0.984877,0.007602,21.051741,0.187058,6.059902e-02,0.002850,-3.167852,0.002429
6,angled,1.0,1.017480,0.019844,3.249812,0.082031,3.814267e-02,0.008261,-3.373860,0.007251
7,angled,70.0,0.989610,0.008937,129.184965,0.880188,1.313190e-02,0.001871,-2.552853,0.001819
8,angled,80.0,0.983183,0.010435,131.108850,1.017533,1.581740e-02,0.002115,-2.491509,0.002147
9,angled,85.0,0.917880,0.007828,131.944408,0.886567,1.638619e-02,0.001812,-2.654894,0.001784


In [57]:
# Same again, but with a plot for floating potential

fig, axes = plt.subplots(2, 3, sharex='col', sharey='col')

for label, s in splopters.items():
    
    if label in skippable:
        continue
    elif 'inpartest' in label or 'flush' in label:
        ax = axes[0][0]
        axt = axes[0][1]
        axd = axes[0][2]
        probe = 'flush'
        ax.set_title('Flush')
    else:
        ax = axes[1][0]
        axt = axes[1][1]
        axd = axes[1][2]
        probe = 'angled'
        ax.set_title('Angled Tip')

    angle = np.abs(float(label[-4:]))
    
    iv_region = np.where(s.iv_data['V'] < -2)
    i = np.where((np.diff(s.iv_data['V'][iv_region]) == 0))
    i = (i[0] + 1, )
    
    if len(i[0]) / len(iv_region[0]) <= 0.25:
        print('Big downsample, skipping...')
    i = iv_region
    
    ax.plot(s.iv_data['V'][i], s.iv_data['I'][i], label=label[-4:])
    ax.legend()
    
    
    filt_df = fit_df.where((fit_df.probe == probe) & (fit_df.angle == angle)).dropna()
    axt.errorbar(filt_df['angle'], filt_df['a'], yerr=filt_df['d_a'], fmt='x')
    axd.errorbar(filt_df['angle'], filt_df['v_f'], yerr=filt_df['d_v_f'], fmt='x')
    

for j, probe in enumerate(['flush', 'angled']):
    
    axt = axes[j][1]
    axd = axes[j][2]
    if j == 0:
        axt.set_title(r'$a$')
        axd.set_title(r'$V_{float}$')
#     axt.errorbar(filt_df['angle'], filt_df['temp'] - 1, yerr=filt_df['d_temp'], fmt='x', ecolor='silver')
#     axt.set_ylim(-0.7, 0.7)
#     axd.errorbar(filt_df['angle'], filt_df['isat'], yerr=filt_df['d_isat'], fmt='x', ecolor='silver')
#     axd.set_ylim(-0.1, 36)
    
fit_df

Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...
Big downsample, skipping...


,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,flush,9.0,0.984877,0.007602,21.051741,0.187058,6.059902e-02,0.002850,-3.167852,0.002429
1,flush,4.0,0.862937,0.021906,8.867943,0.236408,1.305785e-01,0.008518,-3.000390,0.007916
2,flush,7.0,0.940242,0.009109,16.238058,0.168016,7.587003e-02,0.003233,-3.054091,0.002874
3,flush,8.0,1.017323,0.009001,19.388100,0.176069,5.588848e-02,0.002833,-3.014263,0.002248
4,flush,6.0,0.980118,0.021951,14.471464,0.340238,7.609351e-02,0.007440,-3.051633,0.006168
5,flush,5.0,0.984287,0.023889,12.246094,0.301755,8.804362e-02,0.007808,-2.990635,0.006228
6,angled,9.0,1.048546,0.008319,32.368312,0.329622,1.167446e-02,0.003248,-3.362969,0.002814
7,angled,4.0,1.005186,0.017391,14.159134,0.326872,1.674877e-02,0.007322,-3.373216,0.006888
8,angled,10.0,0.928651,0.006629,34.630473,0.261630,2.020882e-02,0.002195,-3.038052,0.002128
9,angled,5.0,1.010071,0.010163,17.942595,0.250309,1.249651e-02,0.004458,-3.457913,0.004251


In [41]:
fig, axes = plt.subplots(3)

for label, s in splopters.items():
    if 'restart' in label:
        ax = axes[0]
        ax.set_title('Flush - Restart')
    elif 'inpartest' in label:
        ax = axes[1]
        ax.set_title('Flush')
    else:
        ax = axes[2]
        ax.set_title('Angled Tip')
    iv_region = np.where(s.iv_data['V'] < 0)
    i = np.where((np.diff(s.iv_data['V'][iv_region]) == 0))
    i = (i[0] + 1, )
    
    ax.plot(s.iv_data['V'][i], s.iv_data['I'][i], label=label)
    ax.legend()

TypeError: 'NoneType' object is not subscriptable

In [42]:
for label, splopter in splopters.items():
    iv_region = np.where(splopter.iv_data['V'] < 0)
    i = np.where((np.diff(splopter.iv_data['V'][iv_region]) == 0))
    
    fig, ax = plt.subplots(2)
    ax[0].set_title(label)
    ax[0].plot(splopter.iv_data['t'][iv_region], splopter.iv_data['V'][iv_region])
    ax[0].plot(splopter.iv_data['t'][i], splopter.iv_data['V'][i], '.')

    ax[1].plot(splopter.iv_data['V'][iv_region], splopter.iv_data['I'][iv_region])
    ax[1].plot(splopter.iv_data['V'][i], splopter.iv_data['I'][i], '.')

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


TypeError: 'NoneType' object is not subscriptable

In [29]:
fit_df = fit_df.sort_values(['probe', 'angle']).reset_index(drop=True)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,2.0,1.045895,0.016761,7.148155,0.163483,1.398953e-02,0.007510,-3.526380,0.006954
1,angled,3.0,1.048542,0.008816,11.100458,0.129524,4.755583e-03,0.003758,-3.484159,0.003447
2,angled,4.0,1.005186,0.017391,14.159134,0.326872,1.674877e-02,0.007322,-3.373216,0.006888
3,angled,5.0,1.010071,0.010163,17.942595,0.250309,1.249651e-02,0.004458,-3.457913,0.004251
4,angled,6.0,0.979680,0.006385,20.553665,0.167514,2.640946e-02,0.002550,-3.281570,0.002369
5,angled,7.0,0.930221,0.006718,23.710272,0.169637,2.973546e-02,0.002070,-2.968275,0.001954
6,angled,8.0,0.888347,0.016929,26.718170,0.539216,3.050687e-02,0.005812,-3.014594,0.005967
7,angled,9.0,1.048546,0.008319,32.368312,0.329622,1.167446e-02,0.003248,-3.362969,0.002814
8,angled,10.0,0.928651,0.006629,34.630473,0.261630,2.020882e-02,0.002195,-3.038052,0.002128
9,angled,10.0,1.033697,0.010810,35.128082,0.458635,1.811715e-02,0.004147,-3.335452,0.003589


## Attempts to get Error on Simulated IV

In [33]:
chosen_spl_path = lowdens_dir / all_run_dirs[scan][2]
print(chosen_spl_path)
splopter = spl.Splopter(chosen_spl_path, reduce=None)
splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0


In [43]:
import scipy.io as sio

print(splopter.tfile_path)

t_dict = sio.loadmat(splopter.tfile_path)
print(t_dict.keys())

/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0/t-alpha_yz_-80.0.mat
dict_keys(['__header__', '__version__', '__globals__', 'version', 'Nz', 'Nzmax', 'Ny', 'count', 'hpos', 'deltah', 'Npc', 'dt', 'dz', 'nproc', 'q', 'm', 'Temp', 'zg', 'yg', 'rho', 'Escz', 'Escy', 'surfacematrix', 'Pot', 'Potvac', 'sliceproc', 'edgecharge', 't', 'snumber', 'totalenergy', 'Esct', 'dPHIqn', 'pchi', 'bx', 'by', 'bz', 'npartproc', 'nodiagreg', 'diaghistories', 'fvarrays', 'fvbin', 'fvperparraycount', 'fvlimits', 'histlimits', 'timehistory_upper', 'timehistory_lower', 'QnPot', 'ProbePot', 'iHistTopx1', 'iHistTopx2', 'iHistTopx3', 'eHistTopx1', 'eHistTopx2', 'eHistTopx3', 'iHistSEx1', 'iHistSEx2', 'iHistSEx3', 'eHistSEx1', 'eHistSEx2', 'eHistSEx3', 'flagm', 'equipotm', 'flag', 'itertime_upper', 'itertime_lower', 'injrate', 'objects', 'edges', 'diagm', 'objectsenum', 'objectscurrenti', 'objectscurrente', 'objectspowerfluxi', 'objectspowerfluxe', 'rho01', 'solw01', 

In [58]:
pfile_paths = [
    '/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0/t-alpha_yz_-80.000.mat',
    '/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0/t-alpha_yz_-80.001.mat',
    '/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0/t-alpha_yz_-80.002.mat',
    '/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0/t-alpha_yz_-80.003.mat'
]
pfile_paths = glob.glob('/home/jleland/Data/external/spice/cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-80.0/t-alpha_yz_-80.0[0-9]*.mat')

npart_procs = []

for pfile_path in pfile_paths:
    variables_of_int = ['ntot']
    p_dict = sio.loadmat(pfile_path, variable_names=variables_of_int)
    npart_procs.append(p_dict['ntot'])
    for label in variables_of_int:
        
        print(f'{label}: {p_dict[label]}')

print(sum(npart_procs))

ntot: [[53300]]
ntot: [[28407]]
ntot: [[0]]
ntot: [[15628]]
ntot: [[61027]]
ntot: [[77137]]
ntot: [[21312]]
ntot: [[691]]
ntot: [[30549]]
ntot: [[89788]]
ntot: [[282826]]
ntot: [[37797]]
ntot: [[9401]]
ntot: [[62289]]
ntot: [[296501]]
ntot: [[294782]]
ntot: [[65075]]
ntot: [[12975]]
ntot: [[96949]]
ntot: [[311204]]
ntot: [[288838]]
ntot: [[169381]]
ntot: [[144515]]
ntot: [[175678]]
ntot: [[321766]]
ntot: [[144653]]
ntot: [[177600]]
ntot: [[195189]]
ntot: [[185533]]
ntot: [[119232]]
ntot: [[0]]
ntot: [[0]]
[[3770023]]


In [30]:
# print(dir(splopter.tdata))

for label in set(non_standard_variables) - {'ProbePot'}:
    print(f'{label}: {splopter.tdata.__dict__[label.lower()]} ({splopter.tdata.__dict__[label.lower()].shape})')

npartproc: [[     0      4     30 ...  53261  53322      0]
 [     0      0      9 ...  28576  28473      0]
 [     0      0      0 ...      0      0      0]
 ...
 [ 10267  13251  16288 ... 118980 118824      0]
 [     0      0      0 ...      0      0      0]
 [     0      0      0 ...      0      0      0]] ((32, 1118))
nproc: [[32]] ((1, 1))
Npc: [[50]] ((1, 1))
Nz: [[138]] ((1, 1))
Nzmax: [[163]] ((1, 1))
Ny: [[557]] ((1, 1))
count: [[71531]] ((1, 1))
snumber: [[    0     0     0 ... 27134 27184     0]
 [    0     4    30 ... 26127 26138     0]] ((2, 1118))


In [61]:
print(splopter.tdata.npartproc.sum())
print(splopter.tdata.snumber.sum())

94703165.04
39845944


In [59]:
N = 138 * 557 * 50
# N = 138 * 557
# N = 50000
print(N)
print(np.sqrt(N) / N)

3843300
0.0005100912303234919


It looks like $Nz \times Ny \times Npc$ is a good measure of the number of particles in the simulation

# Playing with fit_df
A series of cells to plot and handle the data populated into the fit data dataframe

In [75]:
fit_data_filename = 'lowdens_fitdata.csv'

In [76]:
fit_df.to_csv(fit_data_filename)

In [77]:
fit_df = pd.read_csv(fit_data_filename, index_col=0)
fit_df.loc[fit_df.probe == 'flush']

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,flush,4.0,0.862937,0.021906,8.867943,0.236408,1.305785e-01,0.008518,-3.000390,0.007916
1,flush,5.0,0.984287,0.023889,12.246094,0.301755,8.804362e-02,0.007808,-2.990635,0.006228
2,flush,6.0,0.980118,0.021951,14.471464,0.340238,7.609351e-02,0.007440,-3.051633,0.006168
3,flush,7.0,0.940242,0.009109,16.238058,0.168016,7.587003e-02,0.003233,-3.054091,0.002874
4,flush,8.0,1.017323,0.009001,19.388100,0.176069,5.588848e-02,0.002833,-3.014263,0.002248
5,flush,9.0,0.984877,0.007602,21.051741,0.187058,6.059902e-02,0.002850,-3.167852,0.002429
32,flush,10.0,1.011952,0.009473,23.709109,0.223009,5.236428e-02,0.002896,-2.981223,0.002314
33,flush,15.0,1.055231,0.012629,36.369258,0.426445,3.041129e-02,0.003559,-2.932875,0.002739
34,flush,2.0,0.746410,0.041318,3.856090,0.219110,1.897804e-01,0.018324,-2.920495,0.018052
35,flush,20.0,1.035289,0.010473,46.674624,0.443865,3.084954e-02,0.002835,-2.867274,0.002253


In [78]:
fig, axes = plt.subplots(2, 3, sharex='col', sharey='col')
probes = [flush_probe, angled_probe]

for i, tip in enumerate(['flush', 'angled']):
    ax_t = axes[i][0]
    ax_n = axes[i][1]
    ax_vf = axes[i][2]
    probe_df = fit_df.loc[fit_df['probe'] == tip]
    
    dens = probe_df['isat'].values / (lpu.sound_speed(probe_df['temp'].values) * probes[i].get_2d_collection_length(probe_df['angle'].values))
    
    ax_t.errorbar('angle', 'temp', fmt='x', color='r', yerr='d_temp', data=probe_df, label=tip)
    ax_t.axhline(y=1.0, linestyle='dashed', linewidth='0.8', color='gray')
    ax_n.errorbar(probe_df['angle'], dens, fmt='x', color='orange', yerr=probe_df['d_isat'], label=tip)
    ax_vf.errorbar('angle', 'v_f', fmt='x', color='b', yerr='d_v_f', data=probe_df, label=tip)
    
    probe_str = f'{tip} probe \n\n'
    ax_t.set_ylabel(probe_str + r'$T_{e, P}$ / $T_{e, SE}$')
    ax_t.set_xlabel(r'Incidence angle  ($^{\circ}$)')
    ax_t.legend()
    
    ax_n.set_xlabel(r'Incidence angle  ($^{\circ}$)')
    ax_n.set_ylabel(r'$n_{e, P}$ / $n_{e, SE}$')
    ax_n.legend()
    
    ax_vf.set_xlabel(r'Incidence angle  ($^{\circ}$)')
    ax_vf.set_ylabel(r'$V_{f}$')
    ax_vf.legend()
    if i == 0:
        ax_t.set_title('Temperature')
        ax_n.set_title('Density')
        ax_vf.set_title('Floating Potential')

In [80]:
fig, axes = plt.subplots(2, sharex=True, sharey=True)

for i, tip in enumerate(['flush', 'angled']):
    ax_t = axes[i]
    
    probe_df = fit_df.loc[fit_df['probe'] == tip]
    ax_t.errorbar('angle', 'temp', fmt='x', color='r', yerr='d_temp', data=probe_df, label=tip)
    ax_t.axhline(y=1.0, linestyle='dashed', linewidth='0.8', color='gray')
    
    probe_str = f'{tip} probe \n\n'
    ax_t.set_ylabel(probe_str + r'$T_{e, P}$ / $T_{e, SE}$')
    ax_t.set_xlabel(r'Incidence angle  ($^{\circ}$)')
    ax_t.legend()